# Python implementation

In [1]:
# -*- coding: utf-8 -*-
# Implementation by Gilles Van Assche, hereby denoted as "the implementer".
#
# For more information, feedback or questions, please refer to our website:
# https://keccak.team/
#
# To the extent possible under law, the implementer has waived all copyright
# and related or neighboring rights to the source code in this file.
# http://creativecommons.org/publicdomain/zero/1.0/
#
# Code taken from github: https://github.com/XKCP/XKCP

import numpy as np

KECCAK_BYTES = 200
KECCAK_LANES = 25
KECCAK_PLANES_SLICES = 5

THETA_REORDER = ((4, 0, 1, 2, 3), (1, 2, 3, 4, 0))

#Iota Step Round Constants For Keccak-p(1600, 24)
IOTA_CONSTANTS = np.array([0x0000000000000001,0x0000000000008082, 0x800000000000808A,
                            0x8000000080008000, 0x000000000000808B, 0x0000000080000001,
                            0x8000000080008081, 0x8000000000008009, 0x000000000000008A,
                            0x0000000000000088, 0x0000000080008009, 0x000000008000000A,
                            0x000000008000808B, 0x800000000000008B, 0x8000000000008089,
                            0x8000000000008003, 0x8000000000008002, 0x8000000000000080,
                            0x000000000000800A, 0x800000008000000A, 0x8000000080008081,
                            0x8000000000008080, 0x0000000080000001, 0x8000000080008008],
                          dtype=np.uint64)

#Lane Shifts for Rho Step
RHO_SHIFTS = np.array([[0, 36, 3, 41, 18],
                       [1, 44, 10, 45, 2],
                       [62, 6, 43, 15, 61],
                       [28, 55, 25, 21, 56],
                       [27, 20, 39, 8, 14]], dtype=np.uint64)

#Lane Re-order Mapping for Chi Step
CHI_REORDER = ((1, 2, 3, 4, 0), (2, 3, 4, 0, 1))

#Row Re-order Mapping for Pi Step
PI_ROW_REORDER = np.array([[0, 3, 1, 4, 2],
                           [1, 4, 2, 0, 3],
                           [2, 0, 3, 1, 4],
                           [3, 1, 4, 2, 0],
                           [4, 2, 0, 3, 1]])

#Column Re-order Mapping for Pi Step
PI_COLUMN_REORDER = np.array([[0, 0, 0, 0, 0],
                              [1, 1, 1, 1, 1],
                              [2, 2, 2, 2, 2],
                              [3, 3, 3, 3, 3],
                              [4, 4, 4, 4, 4]])


def KeccakF1600(state):
    state = np.copy(np.frombuffer(state, dtype=np.uint64, count=25).reshape([5, 5], order='F'))
    for round_num in range(24):
        # theta_step:
        # Exclusive-or each slice-lane by state based permutation value
        array_shift = state << 1 | state >> 63
        state ^= np.bitwise_xor.reduce(state[THETA_REORDER[0], ], 1, keepdims=True) ^ np.bitwise_xor.reduce(array_shift[THETA_REORDER[1], ], 1, keepdims=True)

        # rho_step:
        # Left Rotate each lane by pre-calculated value
        state = state << RHO_SHIFTS | state >> np.uint64(64 - RHO_SHIFTS)

        # pi_step:
        # Shuffle lanes to pre-calculated positions
        state = state[PI_ROW_REORDER, PI_COLUMN_REORDER]

        # chi_step:
        # Exclusive-or each individual lane based on and/invert permutation
        state ^= ~state[CHI_REORDER[0], ] & state[CHI_REORDER[1], ]

        # iota_step:
        # Exclusive-or first lane of state with round constant
        state[0, 0] ^= IOTA_CONSTANTS[round_num]
    
    return bytearray(state.tobytes(order='F'))

def Keccak(rate, capacity, inputBytes, delimitedSuffix, outputByteLen):
    outputBytes = bytearray()
    state = bytearray([0 for i in range(200)])
    rateInBytes = rate//8
    blockSize = 0
    if (((rate + capacity) != 1600) or ((rate % 8) != 0)):
        return
    inputOffset = 0
    # === Absorb all the input blocks ===
    while(inputOffset < len(inputBytes)):
        blockSize = min(len(inputBytes)-inputOffset, rateInBytes)
        for i in range(blockSize):
            state[i] = state[i] ^ inputBytes[i+inputOffset]
        inputOffset = inputOffset + blockSize
        if (blockSize == rateInBytes):
            state = KeccakF1600(state)
            blockSize = 0
    # === Do the padding and switch to the squeezing phase ===
    state[blockSize] = state[blockSize] ^ delimitedSuffix
    if (((delimitedSuffix & 0x80) != 0) and (blockSize == (rateInBytes-1))):
        state = KeccakF1600(state)
    state[rateInBytes-1] = state[rateInBytes-1] ^ 0x80
    state = KeccakF1600(state)
    # === Squeeze out all the output blocks ===
    while(outputByteLen > 0):
        blockSize = min(outputByteLen, rateInBytes)
        outputBytes = outputBytes + state[0:blockSize]
        outputByteLen = outputByteLen - blockSize
        if (outputByteLen > 0):
            state = KeccakF1600(state)
    return outputBytes

def SHAKE128(inputBytes, outputByteLen):
    return Keccak(1344, 256, inputBytes, 0x1F, outputByteLen)

def SHAKE256(inputBytes, outputByteLen):
    return Keccak(1088, 512, inputBytes, 0x1F, outputByteLen)

def SHA3_224(inputBytes):
    return Keccak(1152, 448, inputBytes, 0x06, 224//8)

def SHA3_256(inputBytes):
    return Keccak(1088, 512, inputBytes, 0x06, 256//8)

def SHA3_384(inputBytes):
    return Keccak(832, 768, inputBytes, 0x06, 384//8)

def SHA3_512(inputBytes):
    return Keccak(576, 1024, inputBytes, 0x06, 512//8)

In [2]:
import struct
import numpy as np
import cbitstruct as bitstruct
from itertools import islice

BITSTRUCT_MATRIX_PACK = "<" + "u64"*256
BITSTRUCT_MATRIX_UNPACK = "<" + "u4"*4096
BITSTRUCT_VECTOR = ">" + "u4"*64

POW_HEADER = struct.pack("<136s", b"\x01\x88\x01\x00\x01\x78ProofOfWorkHash")
HEAVY_HEADER = struct.pack("<136s", b"\x01\x88\x01\x00\x01\x48HeavyHash")

class Xoshiro256PlusPlus(object):
    def __init__(self, state):
        self.state = [x for x in state]

    @staticmethod
    def _rotl(x, k):
        return ((x << k) & 0xFFFFFFFFFFFFFFFF) | (x >> (64 - k))

    def __next__(self):
        result = (self._rotl((self.state[0] + self.state[3]) & 0xFFFFFFFFFFFFFFFF, 23) + self.state[0]) & 0xFFFFFFFFFFFFFFFF

        t = (self.state[1] << 17) & 0xFFFFFFFFFFFFFFFF

        self.state[2] ^= self.state[0]
        self.state[3] ^= self.state[1]
        self.state[1] ^= self.state[2]
        self.state[0] ^= self.state[3]

        self.state[2] ^= t
        self.state[3] = self._rotl(self.state[3], 45)

        return int(result)

    def __iter__(self):
        return self


def calculate_target(bits):
    unshifted_expt = bits >> 24
    if unshifted_expt <= 3:
        mant = (bits & 0xFFFFFF) >> (8 * (3 - unshifted_expt))
        expt = 0
    else:
        mant = bits & 0xFFFFFF
        expt = 8 * ((bits >> 24) - 3)
    return mant << expt


def cast_to_4bit_matrix(buffer):
    return np.array(bitstruct.unpack(BITSTRUCT_MATRIX_UNPACK, buffer), dtype="uint16").reshape(64, 64)


def generate_matrix(header_hash: bytes):
    xoshiro = Xoshiro256PlusPlus(struct.unpack("<4Q", header_hash))

    buffer = bitstruct.pack(BITSTRUCT_MATRIX_PACK, *islice(xoshiro, 256))
    matrix = cast_to_4bit_matrix(buffer)
    while np.linalg.matrix_rank(matrix) < 64:
        buffer = bitstruct.pack(BITSTRUCT_MATRIX_PACK, *islice(xoshiro, 256))
        matrix = cast_to_4bit_matrix(buffer)
    return matrix


def _calculate_hash(header_hash, matrix, timestamp, nonce):
    to_hash = struct.pack("<32sQ32xQ", header_hash, timestamp, nonce)

    # Keccak returns little endian
    pow_hash = Keccak(1088, 512, POW_HEADER + to_hash, 0x04, 32)

    matmul = np.right_shift(np.matmul(matrix, np.array(bitstruct.unpack(BITSTRUCT_VECTOR, pow_hash), dtype="uint16"), dtype="uint16"), 10, dtype="uint16")
    xored = bytes(a^b for (a,b) in zip(pow_hash, bitstruct.pack(BITSTRUCT_VECTOR, *map(int, matmul))))

    # Keccak returns little endian
    heavy_hash = Keccak(1088, 512, HEAVY_HEADER + xored, 0x04, 32)
    return heavy_hash[::-1]

In [3]:
def run_batch(pow_header, matrix, timestamp, target, nonce_start, batch_size=100):
    """
    Mines a batch. Currently uses python as the engine
    Use this function to implement advanced mines
    """
    for i in range(batch_size):
        nonce = nonce_start + i
        
        value = int(_calculate_hash(pow_header, matrix, timestamp, nonce).hex(), 16)
        
        if value < target:
            print("Found block!")
            print("nonce; %d" % nonce)
            print("hash  : %s" % hex(value))
            print("target: %s" % hex(target))
            return

In [4]:
header_hash = b"\x02\x12\x34\x90\x48\x23\x11\x87\x46\x38\x13\x46\x28\x43\x79\x12\x22\x33\x02\x09\x43\x52\x44\x66\x52\x31\x77\x62\x29\x02\x88\x41"
timestamp = 100
target = int("07ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff", 16)
nonce_start = 100

run_batch(header_hash, generate_matrix(header_hash), timestamp, target, nonce_start)

Found block!
nonce; 125
hash  : 0x4ffffaef858637935bbc01179cb397d9080db0dd7ab4350a553029ef02f52a3
target: 0x7ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff


# FPGA implementation

In [5]:
from codecs import encode
from pynq import Overlay, mmio, allocate
import time

In [6]:
overlay = Overlay('/home/xilinx/kaspynq/kaspynq.bit') # load the PL

In [7]:
# PL register map
base_addr = 0x43c00000
version_offset = 0
ctl_offset = 4 # write 1 to bit 0 to start, write 1 to bit 1 to reset, self-clearing
done_offset = 8 # read bit 0 to see if golden nonce was found, self-clearing
target_base_addr_offset = 12
input_data_base_addr_offset = 16
matrix_base_addr_offset = 20
result_base_addr_offset = 24
current_nonce_lsw = 28
current_nonce_msw = 32
starting_nonce_lsw = 36
starting_nonce_msw = 40

reg_space = mmio.MMIO(base_addr, 44)

In [8]:
reg_space.read(version_offset) # check kaspynq version number

27

In [9]:
def allocate_shared_mem():
    #  allocate shared memory regions for target, input data, matrix, and result
    target_buffer = allocate(shape=(32,), dtype='u1') # target, 256 bits, so 32 bytes
    input_buffer = allocate(shape=(72,), dtype='u1') # input data without nonce, 576 bits, so 72 bytes
    matrix_buffer = allocate(shape=(2048,), dtype='u1') # matrix is 64*64*4=16,384 bits, so 2048 bytes
    result_buffer = allocate(shape=(40,), dtype='u1') # 256-bit golden hash result plus 64-bit golden nonce, so 40 bytes
    # sync the result buffer memory to PL
    result_buffer.flush()
    # write addresses of shared memory regions to PL registers
    reg_space.write(target_base_addr_offset, target_buffer.device_address)
    reg_space.write(input_data_base_addr_offset, input_buffer.device_address)
    reg_space.write(matrix_base_addr_offset, matrix_buffer.device_address)
    reg_space.write(result_base_addr_offset, result_buffer.device_address)
    return target_buffer, input_buffer, matrix_buffer, result_buffer

In [10]:
def config_target(target, target_buffer):
    i = 0
    for byte in target:
        target_buffer[i] = byte
        i += 1
    target_buffer.flush()

In [11]:
def config_input(data, input_buffer):
    i = 0
    for byte in data:
        input_buffer[i] = byte
        i += 1
    input_buffer.flush()

In [12]:
def config_matrix(matrix, matrix_buffer):
    for i in range(64):
        for j in range(0,64,2):
            byte = matrix[i][j] + (matrix[i][j+1] << 4)
            matrix_buffer[32*i+j//2] = byte
    matrix_buffer.flush()

In [13]:
def config_nonce_start(nonce_start):
    nonce_start_lsw = nonce_start & 0xffffffff
    nonce_start_msw = (nonce_start >> 32) & 0xffffffff
    reg_space.write(starting_nonce_lsw, nonce_start_lsw)
    reg_space.write(starting_nonce_msw, nonce_start_msw)

In [14]:
target_buffer, input_buffer, matrix_buffer, result_buffer = allocate_shared_mem() # allocate shared memory region, only do once

In [15]:
def run_batch_fpga(pow_header, matrix, target_buffer, matrix_buffer, input_buffer, result_buffer, timestamp, target, nonce_start):
    """
    Mines a batch. Currently uses python as the engine
    Use this function to implement advanced mines
    """
    
    config_matrix(matrix, matrix_buffer)
    to_hash = struct.pack("<32sQ32x", pow_header, timestamp)
    config_input(to_hash, input_buffer)
    config_target(target, target_buffer)
    config_nonce_start(nonce_start)
    reg_space.write(ctl_offset, 1)
    while True:
        done = reg_space.read(done_offset)
        if done == 1:
            break
    result_buffer.invalidate()
    golden_hash = int.from_bytes(result_buffer[0:32], "little")
    golden_nonce = int.from_bytes(result_buffer[32:40], "little")
    print("Found block!")
    print("nonce: %d" % golden_nonce)
    print("hash  : %s" % hex(golden_hash))
    print("target: %s" % target)
    return

In [16]:
header_hash = b"\x02\x12\x34\x90\x48\x23\x11\x87\x46\x38\x13\x46\x28\x43\x79\x12\x22\x33\x02\x09\x43\x52\x44\x66\x52\x31\x77\x62\x29\x02\x88\x41"
timestamp = 100
target = bytes.fromhex("ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff07")
nonce_start = 100

run_batch_fpga(header_hash, generate_matrix(header_hash), target_buffer, matrix_buffer, input_buffer, result_buffer, timestamp, target, nonce_start)

Found block!
nonce: 125
hash  : 0x4ffffaef858637935bbc01179cb397d9080db0dd7ab4350a553029ef02f52a3
target: b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x07'


## Test hashrate

In [17]:
pow_header = b"\x02\x12\x34\x90\x48\x23\x11\x87\x46\x38\x13\x46\x28\x43\x79\x12\x22\x33\x02\x09\x43\x52\x44\x66\x52\x31\x77\x62\x29\x02\x88\x41"
timestamp = 100
target = bytes.fromhex("ffffffffffffffffffffffffffffffffffffffffffffffffffffff0000000000")
matrix = generate_matrix(header_hash)

config_matrix(matrix, matrix_buffer)
to_hash = struct.pack("<32sQ32x", pow_header, timestamp)
config_input(to_hash, input_buffer)
config_target(target, target_buffer)
start = time.time()
reg_space.write(ctl_offset, 1)

In [18]:
reg_space.write(ctl_offset, 2)
end = time.time()
num_hashes = reg_space.read(current_nonce_lsw)
print("%f Mh/s" % (num_hashes / (end-start) / 1000000))

14.995018 Mh/s
